In [33]:
%load_ext autoreload
%autoreload 2
import xml.dom.minidom
import os
from tqdm import tqdm
import glob
import random
import shutil 

import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')
from mitsuba.python.autodiff import render, render_torch, write_bitmap
from mitsuba.python.util import traverse
from mitsuba.core.xml import load_file
from mitsuba.core import Thread, Vector3f, LogLevel



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
# consts
DATASET_FILE_ROOT = '/home/ax/data/radekDataset'
SCENE_FILE_ROOT = f'{DATASET_FILE_ROOT}/scenes'

FRONT_DIR = 'frontalViewRandomColor'
BACK_DIR = 'backViewRandomColor'
TRAIN_SAMPLES = 1800
TEST_SAMPLES = 200
DATA_SET_ROOT = '/home/ax/data/DeepExplain/experiments/data/tshirt'

master_requirements_meshes = ['ground.obj']
master_requirements_texture = ['UV.jpg']


os.makedirs(f'{DATA_SET_ROOT}/train/meshes', exist_ok=True)
os.makedirs(f'{DATA_SET_ROOT}/test/meshes', exist_ok=True)
os.makedirs(f'{DATA_SET_ROOT}/train/texture', exist_ok=True)
os.makedirs(f'{DATA_SET_ROOT}/test/texture', exist_ok=True)
for file in master_requirements_meshes:
    shutil.copy(file, f'{DATA_SET_ROOT}/train/meshes/{file}')
    shutil.copy(file, f'{DATA_SET_ROOT}/test/meshes/{file}')

for file in master_requirements_texture:
    shutil.copy(file, f'{DATA_SET_ROOT}/train/texture/{file}')
    shutil.copy(file, f'{DATA_SET_ROOT}/test/texture/{file}')
    
MASTER_TEMPLATE = 'master_RD.xml'

MASTER_TSHIRT = 'master_tshirt_new_uv.obj'
FACE_AND_UV_COORDS = []

master_file = open(MASTER_TSHIRT, 'r') 
lines = master_file.readlines() 
for line in lines:
    if line.startswith('v '):
        continue
    FACE_AND_UV_COORDS.append(line)

# scale should be min -7 
Z_MIN = 7
Z_MAX = 7

In [35]:
def get_scenes_filenames(directory):
    scenes = []
    for file in tqdm(glob.glob(f"{directory}/*.xml")):
        scenes.append(file)
    # shuffel scenes due to naming 
    random.seed(42)
    random.shuffle(scenes)
    
    return scenes

def copy_character_mesh(file_name, train_flag):
    t_dir = 'train' if train_flag else 'test'
    # /cluster/scratch/rdanecek/arcsim/SPRING_MALE_RESAMPLED/mesh50_animated/127/127_10/frame_00144.obj
    local_path = file_name.replace('/cluster/scratch/rdanecek/arcsim/', '')
    src_path = f'{DATASET_FILE_ROOT}/{local_path}'
    
    new_path = f'meshes/{local_path}'
    trg_path = f'{DATA_SET_ROOT}/{t_dir}/{new_path}'
    
    # create all dirs 
    os.makedirs(os.path.dirname(trg_path), exist_ok=True)
    if not os.path.isfile(src_path):
        print(f'file: {src_path} does not exist.')
        return
    shutil.copy(src_path, trg_path)
    
    return new_path

def update_thsirt_uv_map(file_name):
    tshirt_file = open(file_name, 'r') 
    lines = tshirt_file.readlines()
    tshirt_file.close() 
    
    vertex_positions = []
    
    for line in lines:
        if line.startswith('v '):
            vertex_positions.append(line)
    
    content = vertex_positions + FACE_AND_UV_COORDS
    
    tshirt_file = open(file_name, 'w') 
    tshirt_file.writelines(content) 
    tshirt_file.close() 
    
    
    
def copy_tshirt_mesh(file_name, train_flag):
    t_dir = 'train' if train_flag else 'test'
    
    local_path = file_name.replace('/cluster/scratch/rdanecek/arcsim/', '')
    src_path = f'{DATASET_FILE_ROOT}/{local_path}'
    
    new_path = f'meshes/{local_path}'
    trg_path = f'{DATA_SET_ROOT}/{t_dir}/{new_path}'
    
    # create all dirs 
    os.makedirs(os.path.dirname(trg_path), exist_ok=True)
    if not os.path.isfile(src_path):
        print(f'file: {src_path} does not exist.')
        return
    
    shutil.copy(src_path, trg_path)
    
    update_thsirt_uv_map(trg_path)
    
    return new_path

def get_scene_info(scene_file_name, front_flag, train_flag):
    scene_doc = xml.dom.minidom.parse(scene_file_name)
    master_doc = xml.dom.minidom.parse(MASTER_TEMPLATE)
    
    # get the camera node
    src_camera = scene_doc.getElementsByTagName('sensor')
    trg_camera = master_doc.getElementsByTagName('sensor')
    if len(src_camera) != 1:
        print('No sensor found')
        return
    src_camera_transform = src_camera[0].getElementsByTagName('transform')[0]
    trg_camera_transform = trg_camera[0].getElementsByTagName('transform')[0]
    
    # move a bit more back
    src_look_at = src_camera_transform.getElementsByTagName('lookat')[0]
    origin =  src_look_at.getAttribute('origin').split(',')
    origin = [float(i) for i in origin]
    origin[2] += random.uniform(Z_MIN, Z_MAX) if front_flag else -1 * (random.uniform(Z_MIN, Z_MAX)) 
    origin = [str(i) for i in origin]
    src_look_at.setAttribute('origin', ','.join(origin))
    
    trg_camera[0].replaceChild(src_camera_transform, trg_camera_transform)
    
    # get character transform
    src_shapes = scene_doc.getElementsByTagName('shape')
    trg_shapes = master_doc.getElementsByTagName('shape')
    
    src_character = None
    trg_character = None
    
    src_tshirt = None
    trg_tshirt = None
    
    for i in range(len(src_shapes)):
        if src_shapes[i].getAttribute("id") == 'character':
            src_character = src_shapes[i]
        if src_shapes[i].getAttribute("id") == 'simulated':
            src_tshirt = src_shapes[i]
            
    for i in range(len(trg_shapes)):
        if trg_shapes[i].getAttribute("id") == 'character':
            trg_character = trg_shapes[i]
        if trg_shapes[i].getAttribute("id") == 'simulated':
            trg_tshirt = trg_shapes[i]
            
    if src_character == None:
        print('Character not in scene file')
        return
    
    if src_tshirt == None:
        print('Tshirt not in scene file')
        return
    
    src_char_transform = src_character.getElementsByTagName('transform')[0]
    trg_char_transform = trg_character.getElementsByTagName('transform')[0]
    trg_character.replaceChild(src_char_transform, trg_char_transform)
    
    src_tshirt_transform = src_tshirt.getElementsByTagName('transform')[0]
    trg_tshirt_transform = trg_tshirt.getElementsByTagName('transform')[0]
    trg_tshirt.replaceChild(src_tshirt_transform, trg_tshirt_transform)
    
    # meshes
    src_char_filename_node = src_character.getElementsByTagName('string')[0]
    char_filename = src_char_filename_node.getAttribute('value')
    
    trg_char_filename_node = trg_character.getElementsByTagName('string')[0]
    new_char_filename = copy_character_mesh(char_filename, train_flag)
    trg_char_filename_node.setAttribute('value', new_char_filename)
    
    # Tshirt
    src_tshirt_filename_node = src_tshirt.getElementsByTagName('string')[0]
    tshirt_filename = src_tshirt_filename_node.getAttribute('value')
    
    trg_tshirt_filename_node = trg_tshirt.getElementsByTagName('string')[0]
    new_tshirt_filename = copy_tshirt_mesh(tshirt_filename, train_flag)
    trg_tshirt_filename_node.setAttribute('value', new_tshirt_filename)
    
    return master_doc.toxml()
    
    
def save_scene(scene_str, i, path):
    file_name = f'{path}/{i:05d}.xml'
    with open(file_name, "w") as f:
        f.write(scene_str)
    return file_name

def render_file(file_name):
    thread = Thread.thread()
    thread.file_resolver().append(os.path.dirname(file_name))
    logger = thread.logger()
    logger.set_log_level(LogLevel.Warn)

    scene = load_file(file_name)
    image_ref = render(scene, spp=32)
    crop_size = scene.sensors()[0].film().crop_size()
    out_file_name = file_name.replace('xml', 'png')
    write_bitmap(out_file_name, image_ref, crop_size)
        

In [37]:
front_scenes = get_scenes_filenames(f'{SCENE_FILE_ROOT}/{FRONT_DIR}')
back_scenes = get_scenes_filenames(f'{SCENE_FILE_ROOT}/{BACK_DIR}')

100%|██████████| 100000/100000 [00:00<00:00, 5077235.20it/s]


In [38]:
print('Generate training data')
for i in tqdm(range(TRAIN_SAMPLES)):
    scene_file_path = front_scenes.pop(0) if i % 2 == 0 else back_scenes.pop(0)
    scene_str =  get_scene_info(scene_file_path, i % 2 == 0, True)
    new_scene_file_path = save_scene(scene_str, i, f'{DATA_SET_ROOT}/train')
    render_file(new_scene_file_path)
    

  0%|          | 0/1800 [00:00<?, ?it/s]

Generate training data


100%|██████████| 1800/1800 [08:56<00:00,  3.35it/s]


In [39]:
print('Generate test data')
for i in tqdm(range(TEST_SAMPLES)):
    scene_file_path = front_scenes.pop(0) if i % 2 == 0 else back_scenes.pop(0)
    scene_str =  get_scene_info(scene_file_path, i % 2 == 0, False)
    new_scene_file_path = save_scene(scene_str, i, f'{DATA_SET_ROOT}/test')
    render_file(new_scene_file_path)

  0%|          | 0/200 [00:00<?, ?it/s]

Generate test data


100%|██████████| 200/200 [01:24<00:00,  2.38it/s]
